In [117]:
import pandas as pd
import missingno as msno
import matplotlib.pyplot as plt
import pandas as pd
import requests
import time
import numpy as np
import re
import seaborn as sns
from sklearn.model_selection import train_test_split as sklearn_train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from surprise import Reader, Dataset, SVD
from surprise import accuracy
import random


In [118]:
books = pd.read_csv('/home/administrator/Desktop/IDE/Recomendation/books_new.csv')
ratings = pd.read_csv('/home/administrator/Desktop/IDE/Recomendation/ratings.csv')
tags =  pd.read_csv('/home/administrator/Desktop/IDE/Recomendation/tags.csv')
book_tags = pd.read_csv('/home/administrator/Desktop/IDE/Recomendation/book_tags.csv')

## ФУНКЦИЯ РАЗДЕЛЕНИЯ

In [119]:
def split_random_ratings(ratings, test_size=0.2):
    """
    Случайное разделение датасета рейтингов
    """
    print("Выполняется случайное разделение данных...")
    start_time = time.time()
    train, test = sklearn_train_test_split(ratings, test_size=test_size, random_state=42)
    elapsed = time.time() - start_time
    return train, test, elapsed

## ФУНКЦИЯ ОЦЕНКИ

In [120]:
def optimized_evaluate_metrics(recommender, test_data, user_seen_items_dict, all_items, k=10):
    """Оценка Top-K рекомендаций по метрикам."""
    test_user_items = test_data.groupby('user_id')['book_id'].agg(list).to_dict()
    catalog_size = len(all_items)

    precision_sum = 0
    recall_sum = 0
    ndcg_sum = 0
    arp_sum = 0
    hit_rate_sum = 0
    catalog_coverage = set()
    users_evaluated = 0

    for user_id, true_items in test_user_items.items():
        if not true_items:
            continue

        user_history = user_seen_items_dict.get(user_id, set())
        
        # для SVD
        if hasattr(recommender, 'requires_user_id') and recommender.requires_user_id:
            # Для SVD
            recs = recommender.recommend(user_id, user_history, k=k)
        else:
            # Для Popularity, Content-Based: Передаем только user_history и k
            recs = recommender.recommend(user_history, k=k)
        # ------------------------------------------------------------------------

        if not recs:
            continue

        users_evaluated += 1
        catalog_coverage.update(recs)

        rec_set = set(recs)
        true_set = set(true_items)
        hits = rec_set & true_set
        num_hits = len(hits)

        # Precision и Recall
        precision = num_hits / len(recs)
        recall = num_hits / len(true_set)

        precision_sum += precision
        recall_sum += recall
        hit_rate_sum += 1 if num_hits > 0 else 0

        # nDCG
        relevance = [1 if item in true_set else 0 for item in recs]
        if num_hits > 0:
            ideal = sorted(relevance, reverse=True)
            dcg = sum(r / np.log2(i + 2) for i, r in enumerate(relevance))
            idcg = sum(r / np.log2(i + 2) for i, r in enumerate(ideal))
            ndcg = dcg / idcg
        else:
            ndcg = 0
        ndcg_sum += ndcg

        # ARP
        arp_score = sum(1 / (i + 1) for i, item in enumerate(recs) if item in true_set)
        arp_sum += arp_score / len(recs)

    coverage = len(catalog_coverage) / catalog_size if catalog_size > 0 else 0

    return {
        'precision@10': precision_sum / users_evaluated if users_evaluated > 0 else 0,
        'recall@10': recall_sum / users_evaluated if users_evaluated > 0 else 0,
        'ndcg@10': ndcg_sum / users_evaluated if users_evaluated > 0 else 0,
        'arp@10': arp_sum / users_evaluated if users_evaluated > 0 else 0,
        'hit_rate@10': hit_rate_sum / users_evaluated if users_evaluated > 0 else 0,
        'coverage': coverage,
        'users_evaluated': users_evaluated
    }

print("\n Оптимизированная функция оценки настроена (с поддержкой ID-моделей).")


 Оптимизированная функция оценки настроена (с поддержкой ID-моделей).


##  МОДЕЛЬ 1: МОДЕЛЬ ОСНОВАННАЯ НА ПОПУЛЯРНОСТИ

**Эта модель рекомендует всем пользователям одни и те же популярные книги. Под "популярными" мы будем понимать книги с высоким средним рейтингом и значительным количеством оценок для обеспечения надежности.**

In [83]:
train, test, split_time = split_random_ratings(ratings, test_size=0.2)

print(f"\nФИНАЛЬНОЕ РАЗДЕЛЕНИЕ (Случайное):")
print(f"  train: {len(train):,} записей")
print(f"  test:  {len(test):,} записей")

Выполняется случайное разделение данных...

ФИНАЛЬНОЕ РАЗДЕЛЕНИЕ (Случайное):
  train: 4,781,183 записей
  test:  1,195,296 записей


In [60]:
all_items = ratings['book_id'].unique()

# История просмотров/оценок по пользователям (из train)
user_seen_items = train.groupby('user_id')['book_id'].agg(set).to_dict()

# Оставляем только пользователей, которые есть и в train, и в test (для корректной оценки)
common_users = set(train['user_id']) & set(test['user_id'])
test_filtered = test[test['user_id'].isin(common_users)].copy()

print(f"  Пользователей для оценки: {len(common_users):,}")

  Пользователей для оценки: 53,423


**Мы оцениваем только на 53,423 пользователях потому что:**

* У них есть история в train (можно строить рекомендации)

* У них есть реальные взаимодействия в test (можно измерить качество)

In [61]:
class PopularityRecommender:
    def __init__(self):
        self.popular_books = None

    def fit(self, train_data):
        print("Обучение Popularity-модели...")
        # Сортируем по популярности (количеству оценок)
        self.popular_books = train_data['book_id'].value_counts().index.tolist()

    def recommend(self, user_seen_books, k=10):
        if not self.popular_books:
            return []
        seen_set = set(user_seen_books)
        recs = []
        for book in self.popular_books:
            if book not in seen_set:
                recs.append(book)
                if len(recs) >= k:
                    break
        return recs

# ------------------------------------------------------------
# 6. Обучение и оценка
# ------------------------------------------------------------
print("\nОбучение модели...")
start_time = time.time()
pop_model = PopularityRecommender()
pop_model.fit(train)
train_time = time.time() - start_time

print("Оценка модели...")
start_time = time.time()
metrics = optimized_evaluate_metrics(pop_model, test_filtered, user_seen_items, all_items, k=10)
eval_time = time.time() - start_time

# ------------------------------------------------------------
# 7. Вывод результатов
# ------------------------------------------------------------
print("\nРЕЗУЛЬТАТЫ POPULARITY МОДЕЛИ:")
print(f"  NDCG@10:    {metrics['ndcg@10']:.4f}")
print(f"  Precision@10: {metrics['precision@10']:.4f}")
print(f"  Recall@10:    {metrics['recall@10']:.4f}")
print(f"  ARP@10:       {metrics['arp@10']:.4f}")
print(f"  HitRate@10:   {metrics['hit_rate@10']:.2%}")
print(f"  Coverage:     {metrics['coverage']:.2%}")
print(f"  Время обучения: {train_time:.1f} сек")
print(f"  Время оценки:   {eval_time:.1f} сек")

# Сохраняем результаты
results = [{
    'Алгоритм': 'Popularity',
    'NDCG@10': metrics['ndcg@10'],
    'Precision@10': metrics['precision@10'],
    'Recall@10': metrics['recall@10'],
    'ARP@10': metrics['arp@10'],
    'HitRate@10': metrics['hit_rate@10'],
    'Coverage': metrics['coverage'],
    'Время_обучения': train_time,
    'Время_оценки': eval_time
}]


Обучение модели...
Обучение Popularity-модели...
Оценка модели...

РЕЗУЛЬТАТЫ POPULARITY МОДЕЛИ:
  NDCG@10:    0.3181
  Precision@10: 0.1019
  Recall@10:    0.0456
  ARP@10:       0.0363
  HitRate@10:   51.47%
  Coverage:     0.44%
  Время обучения: 0.3 сек
  Время оценки:   3.2 сек


* Hit Rate@10 = 51.47%
Popularity-модель ничего не знает о пользователе — она просто выдаёт общие хиты. То, что она "попадает" в интересы половины пользователей — уже неплохо.

* Precision@10 = 10.19%
Из 10 рекомендованных книг в среднем 1 книга окажется релевантной.

* . Recall@10 = 4.56%
Recall низкий — потому что большинство релевантных книг пользователя находятся в "длинном хвосте", а Popularity их не видит. 

* NDCG@10 = 0.3181
одель умеет ранжировать (популярные книги действительно чаще релевантны), но не идеально.

* Coverage = 0.44%
Пользователи видят одни и те же книги.Нет разнообразия, нет открытий.
Популярное становится ещё популярнее, и это так называемый эффект Мэтью.

*   ARP@10 = 0.0363
Если релевантная книга на 1-м месте → вклад = 1/1 = 1.0

**Эта модель проста и эффективна для новых пользователей (холодный старт), но ей не хватает персонализации. Popularity-модель является самой быстрой и масштабируемой в обучении и генерации рекомендаций. Эта модель очень хороша как бейзлайн. Все последующие персонализированные модели (CF, SVD, Content-Based) должны показывать значимо лучшие результаты по метрикам Precision/Recall/NDCG, чтобы считаться успешными.**

## Модель 2: Content-Based Model

Эта модель рекомендует книги, похожие на данную книгу, на основе ее содержания (названия и тегов).

Подготовка данных


In [89]:
book_tags_with_names = pd.merge(book_tags, tags, on='tag_id')

book_tags_agg = book_tags_with_names.groupby('goodreads_book_id')['tag_name'].apply(lambda x: ' '.join(x)).reset_index() # Агрегируем все теги по книге в одну строку

books_with_tags = pd.merge(books, book_tags_agg, on='goodreads_book_id', how='left') # Присоединяем к книгам
books_with_tags['tag_name'] = books_with_tags['tag_name'].fillna('')

books_with_tags['content'] = books_with_tags['original_title'].fillna('') + ' ' + books_with_tags['tag_name'] # Создаём контентный профиль

books_with_tags = books_with_tags.set_index('book_id') # Устанавливаем book_id как индекс для удобства

print(f"  → Подготовлено {len(books_with_tags)} книг с контентными профилями.")

#  Векторизация и матрица схожести

print("2. TF-IDF векторизация и расчёт схожести...")

tfidf = TfidfVectorizer(stop_words='english', max_features=5000)
tfidf_matrix = tfidf.fit_transform(books_with_tags['content'])

cosine_sim = cosine_similarity(tfidf_matrix)

# Создаём маппинги между book_id и индексом в матрице
book_id_to_idx = {book_id: idx for idx, book_id in enumerate(books_with_tags.index)}
idx_to_book_id = {idx: book_id for book_id, idx in book_id_to_idx.items()}

print("  → Матрица схожести готова.")

  → Подготовлено 10000 книг с контентными профилями.
2. TF-IDF векторизация и расчёт схожести...
  → Матрица схожести готова.


In [51]:
class ContentBasedRecommender:
    def __init__(self, books_with_tags, cosine_sim, book_id_to_idx, idx_to_book_id):
        self.books_with_tags = books_with_tags
        self.cosine_sim = cosine_sim
        self.book_id_to_idx = book_id_to_idx
        self.idx_to_book_id = idx_to_book_id

    def recommend(self, user_seen_books, k=10):
        """
        Генерирует рекомендации на основе истории пользователя.
        Берёт последнюю книгу из истории как основу и находит похожие.
        """
        # Фильтруем только те книги, которые есть в нашем контентном датасете
        seen_in_catalog = [bid for bid in user_seen_books if bid in self.book_id_to_idx]
        if not seen_in_catalog:
            return []

        # Берём последнюю книгу как основу
        seed_book_id = seen_in_catalog[-1]

        idx = self.book_id_to_idx[seed_book_id]
        sim_scores = list(enumerate(self.cosine_sim[idx]))
        # Сортируем по убыванию, пропускаем первую (саму себя)
        sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)[1:]

        recs = []
        seen_set = set(user_seen_books)
        for i, score in sim_scores:
            book_id = self.idx_to_book_id[i]
            if book_id not in seen_set:
                recs.append(book_id)
                if len(recs) >= k:
                    break
        return recs

# ------------------------------------------------------------
# 4. Создание и оценка модели
# ------------------------------------------------------------
print("\n3. Создание и оценка модели...")

content_model = ContentBasedRecommender(
    books_with_tags, cosine_sim, book_id_to_idx, idx_to_book_id
)

# Оценка
start_time = time.time()
metrics_content = optimized_evaluate_metrics(
    content_model, test_filtered, user_seen_items, all_items, k=10
)
eval_time = time.time() - start_time

# ------------------------------------------------------------
# 5. Вывод результатов
# ------------------------------------------------------------
print("\nРЕЗУЛЬТАТЫ CONTENT-BASED МОДЕЛИ:")
print(f"  NDCG@10:    {metrics_content['ndcg@10']:.4f}")
print(f"  Precision@10: {metrics_content['precision@10']:.4f}")
print(f"  Recall@10:    {metrics_content['recall@10']:.4f}")
print(f"  ARP@10:       {metrics_content['arp@10']:.4f}")
print(f"  HitRate@10:   {metrics_content['hit_rate@10']:.2%}")
print(f"  Coverage:     {metrics_content['coverage']:.2%}")
print(f"  Время оценки: {eval_time:.1f} сек")

# Добавляем в общий список результатов (если есть)
try:
    results.append({
        'Алгоритм': 'Content-Based',
        'NDCG@10': metrics_content['ndcg@10'],
        'Precision@10': metrics_content['precision@10'],
        'Recall@10': metrics_content['recall@10'],
        'ARP@10': metrics_content['arp@10'],
        'HitRate@10': metrics_content['hit_rate@10'],
        'Coverage': metrics_content['coverage'],
        'Время_обучения': 0,  # обучение = подготовка матрицы (уже сделано)
        'Время_оценки': eval_time
    })
    print("\n Результаты добавлены в общий список.")
except NameError:
    print("\n Переменная 'results' не определена. Создаём новую.")
    results = [{
        'Алгоритм': 'Content-Based',
        'NDCG@10': metrics_content['ndcg@10'],
        'Precision@10': metrics_content['precision@10'],
        'Recall@10': metrics_content['recall@10'],
        'ARP@10': metrics_content['arp@10'],
        'HitRate@10': metrics_content['hit_rate@10'],
        'Coverage': metrics_content['coverage'],
        'Время_обучения': 0,
        'Время_оценки': eval_time
    }]


  → Подготовлено 10000 книг с контентными профилями.
2. TF-IDF векторизация и расчёт схожести...
  → Матрица схожести готова.

3. Создание и оценка модели...

РЕЗУЛЬТАТЫ CONTENT-BASED МОДЕЛИ:
  NDCG@10:    0.2579
  Precision@10: 0.0685
  Recall@10:    0.0324
  ARP@10:       0.0307
  HitRate@10:   37.89%
  Coverage:     75.74%
  Время оценки: 408.3 сек

 Результаты добавлены в общий список.


**Сильные стороны**

Огромный Coverage (75.7%)
→ Модель работает с "длинным хвостом", рекомендует нишевые книги, а не только бестселлеры.
→ Это главное преимущество перед Popularity!
Персонализация по содержанию
→ Если пользователь читает "японскую литературу", модель предложит похожие книги, даже если они не популярны. 

Слабые стороны
Низкие Precision, Recall, Hit Rate
→ Контентные профили недостаточно точны. Причины:
Шумные теги: много "to-read", "favorites" вместо жанров.
Короткие профили: у многих книг мало тегов или только общие ("fiction").
Нет учёта пользовательских предпочтений: модель не знает, нравится ли пользователю жанр — она просто ищет похожие книги.
Хуже ранжирование (NDCG)
→ Даже когда релевантные книги есть в рекомендациях, они не в топе списка.

Контентная модель имеет высокую предварительную сложность O(N**2) и требует значительных ресурсов памяти, но после подготовки генерация рекомендаций для одного пользователя работает эффективно.

В данном коде взята за основу  Content-Based на основе последней активности (Last-Item Recommendation). Почему выбран именно этот подход?

* 1. Быстродействие. Модель крайне быстра в режиме рекомендаций. Поскольку матрица схожести N×N рассчитана заранее, генерация рекомендаций сводится к одной операции поиска по индексу (вместо сложной агрегации профиля)
* 2. Актуальность. Пользовательские предпочтения постоянно меняются. Использование самого последнего прочитанного элемента (книги-основы) фокусирует рекомендательную систему на текущем, сиюминутном интересе пользователя, игнорируя его давнюю историю.
* 3. Вычислительная эффективность. Подход позволяет избежать построения сложного профиля пользователя (агрегации TF-IDF векторов всех прочитанных книг), что существенно экономит вычислительные ресурсы и время, особенно если у пользователя очень длинная история (тысячи прочитанных книг).

Слабая сторона: Главный недостаток — низкое качество (NDCG) и низкая точность. Модель игнорирует всю остальную историю и все оценки пользователя. Мы отказались от создания взвешенного профиля пользователя , который бы учитывал все прочитанные книги и их оценки, чтобы повысить NDCG и Precision из-за вычислительного огранияения. Взвешенный профиль требует агрегации (суммирования) TF-IDF векторов для всех прочитанных книг каждого пользователя.


## Модель 3. Collaborative Filtering (CF)

In [ ]:
# Создаём user-item матрицу
print("1. Создание user-item матрицы...")
user_item_matrix = ratings.pivot_table(index='user_id', columns='book_id', values='rating')
user_item_matrix.fillna(0, inplace=True)  # 0 = отсутствие оценки
print("2. Расчёт схожести между книгами...")



In [ ]:
# Транспонируем: книги → строки
item_similarity = cosine_similarity(user_item_matrix.T)
item_sim_df = pd.DataFrame(
    item_similarity,
    index=user_item_matrix.columns,   # book_id
    columns=user_item_matrix.columns  # book_id
)

In [52]:
class ItemBasedRecommender:
    def __init__(self, item_sim_df):
        self.item_sim_df = item_sim_df

    def recommend(self, user_seen_books, k=10):
        """
        Генерирует рекомендации на основе истории пользователя.
        Для каждой книги пользователя находит похожие и агрегирует схожесть.
        """
        if not user_seen_books:
            return []

        # Оставляем только книги, которые есть в матрице схожести
        seen_in_catalog = [bid for bid in user_seen_books if bid in self.item_sim_df.index]
        if not seen_in_catalog:
            return []

        # Агрегируем схожесть: суммируем scores по всем просмотренным книгам
        similarity_scores = pd.Series(dtype='float64')
        for book_id in seen_in_catalog:
            sim_scores = self.item_sim_df[book_id]
            similarity_scores = similarity_scores.add(sim_scores, fill_value=0)

        # Исключаем уже просмотренные книги
        similarity_scores = similarity_scores.drop(seen_in_catalog, errors='ignore')

        # Берём топ-k
        top_books = similarity_scores.nlargest(k)
        return top_books.index.tolist()

# 4. Оценка модели
print("\n3. Создание и оценка модели...")

item_model = ItemBasedRecommender(item_sim_df)

start_time = time.time()
metrics_item = optimized_evaluate_metrics(
    item_model, test_filtered, user_seen_items, all_items, k=10
)
eval_time = time.time() - start_time

# 5. Вывод результатов
print("\nРЕЗУЛЬТАТЫ ITEM-BASED CF МОДЕЛИ:")
print(f"  NDCG@10:    {metrics_item['ndcg@10']:.4f}")
print(f"  Precision@10: {metrics_item['precision@10']:.4f}")
print(f"  Recall@10:    {metrics_item['recall@10']:.4f}")
print(f"  ARP@10:       {metrics_item['arp@10']:.4f}")
print(f"  HitRate@10:   {metrics_item['hit_rate@10']:.2%}")
print(f"  Coverage:     {metrics_item['coverage']:.2%}")
print(f"  Время оценки: {eval_time:.1f} сек")

# 6. Добавляем в результаты
results.append({
    'Алгоритм': 'Item-Based CF',
    'NDCG@10': metrics_item['ndcg@10'],
    'Precision@10': metrics_item['precision@10'],
    'Recall@10': metrics_item['recall@10'],
    'ARP@10': metrics_item['arp@10'],
    'HitRate@10': metrics_item['hit_rate@10'],
    'Coverage': metrics_item['coverage'],
    'Время_обучения': 0,  # обучение = расчёт матрицы (уже сделано)
    'Время_оценки': eval_time
})

print("\nItem-Based CF успешно оценена!")


=== ITEM-BASED COLLABORATIVE FILTERING ===
1. Создание user-item матрицы...
2. Расчёт схожести между книгами...
  → Матрица схожести книг готова.

3. Создание и оценка модели...

РЕЗУЛЬТАТЫ ITEM-BASED CF МОДЕЛИ:
  NDCG@10:    0.6305
  Precision@10: 0.2738
  Recall@10:    0.1281
  ARP@10:       0.1025
  HitRate@10:   85.89%
  Coverage:     29.10%
  Время оценки: 1553.8 сек

✅ Item-Based CF успешно оценена!


* NDCG@10 - 0.6305 ----Показывает, что релевантные книги отлично ранжируются в топе списка.
* Precision@10 - 0.2738 -----Высокая точность. В среднем, 2.7 книги из 10 будут релевантны.
* Recall@10 -0.1281 -Низкий Recall. Типично для Top-K. Это означает, что модель находит только ≈12.8% всех релевантных книг пользователя в тестовом наборе.
* HitRate@10 - 85.89% -Отличный результат. Почти для 86% пользователей модель успешно поймала хотя бы одну релевантную книгу в топ-10.
* Coverage -29.10% Низкий Coverage. Модель рекомендует только ≈29% уникальных книг из каталога. Это классическая проблема Item-Based CF: она рекомендует только те книги, которые похожи на популярные книги, что снижает разнообразие.
* Время оценки - Очень медленно. Основная проблема модели в оценке и обучении.

Вычислительная сложность: Вычисление полной матрицы подобия элементов составляет O(n 2⋅m), где n - количество элементов, а m - количество пользователей. Это очень дорого для больших наборов данных.
Оптимизация: Для больших данных вместо вычисления сходства для всех пар мы можем использовать такие методы, как хеширование с учетом местоположения (LSH), чтобы найти подходящие пары похожих элементов, или мы можем предварительно рассчитать матрицу в автономном режиме и периодически обновлять ее. Если Content-Based модель показывает NDCG ≈0.5, то Item-Based CF (NDCG ≈0.63) значительно лучше в плане качества, но значительно хуже в плане времени.

## Модель 4. Matrix Factorization (SVD)

SVD масштабируема и, как правило, предоставляет более точные, персонализированные рекомендации, чем базовая CF.

In [123]:
start_time_train = time.time()

# 1. Подготовка данных для surprise
reader = Reader(rating_scale=(1, 5))
# Используем ВЕСЬ датасет ratings для создания полного маппинга ID
data = Dataset.load_from_df(ratings[['user_id', 'book_id', 'rating']], reader)

# Преобразуем train в формат surprise, чтобы обучать только на train
train_tuples = [(row.user_id, row.book_id, row.rating, None) for row in train.itertuples()]
trainset = data.construct_trainset(train_tuples)

# 2. Обучение SVD
print("Обучение SVD модели...")
svd_model = SVD(n_factors=50, n_epochs=20, random_state=42)
svd_model.fit(trainset)
train_time_svd = time.time() - start_time_train
print(f"SVD модель обучена за {train_time_svd:.1f} сек.")

# 3. Оценка RMSE (для классического сравнения)
testset_surprise = [(row.user_id, row.book_id, row.rating) for row in test_filtered.itertuples()]
predictions_rmse = svd_model.test(testset_surprise)
print(f"SVD Model RMSE on test set: {accuracy.rmse(predictions_rmse):.4f}")

Обучение SVD модели...
SVD модель обучена за 90.7 сек.
RMSE: 0.8280
SVD Model RMSE on test set: 0.8280


In [124]:
class SVDRecommender:
    """
    ОПТИМИЗИРОВАННАЯ ВЕРСИЯ: Прогнозирует только по ТОП-2000 популярных книг (Item Sampling),
    что значительно ускоряет оценку Top-K метрик.
    """
    # ФЛАГ: Говорит optimized_evaluate_metrics, что нам нужен user_id
    requires_user_id = True 
    
    def __init__(self, svd_model, train_data, all_items):
        self.svd_model = svd_model
        self.all_items = list(all_items)
        
        # *** ОПТИМИЗАЦИЯ 1: Подмножество книг для прогнозирования (ТОП-2000)
        self.popular_books_subset = train_data['book_id'].value_counts().index.tolist()[:2000] 
        
        self.popular_books = train_data['book_id'].value_counts().index.tolist()
        self.known_users = set(train_data['user_id'].unique())
        
    def recommend(self, user_id, user_seen_books, k=10):
        user_seen_set = set(user_seen_books)
        
        # 1. Обработка холодных пользователей
        if user_id not in self.known_users:
            recs = [b for b in self.popular_books if b not in user_seen_set]
            return recs[:k]
            
        # 2. Генерация рекомендаций через SVD (ТОЛЬКО по подмножеству)
        books_to_predict = [
            bid for bid in self.popular_books_subset 
            if bid not in user_seen_set
        ]
        
        if not books_to_predict:
            # Если нет книг для прогноза, возвращаем пустой список
            return []
            
        # Делаем предсказания (фиктивный рейтинг 4.0)
        testset_for_user = [[user_id, book_id, 4.0] for book_id in books_to_predict]
        predictions = self.svd_model.test(testset_for_user)
        predictions.sort(key=lambda x: x.est, reverse=True)
        recs = [pred.iid for pred in predictions[:k]]
        
        return recs


# Cell: Оценка SVD и вывод метрик (ОПТИМИЗИРОВАННЫЙ)
print("\n--- 3. ОЦЕНКА SVD ПО TOP-K МЕТРИКАМ (ОПТИМИЗИРОВАННЫЙ РАСЧЕТ) ---\n")

svd_recommender = SVDRecommender(
    svd_model=svd_model,
    train_data=train,
    all_items=ratings['book_id'].unique(),
)

# *** ОПТИМИЗАЦИЯ 2: Сэмплирование пользователей (User Sampling) ***
SAMPLE_SIZE = 10000 
users_to_sample = list(test_filtered['user_id'].unique())

if len(users_to_sample) > SAMPLE_SIZE:
    # Создаем сэмпл пользователей
    sampled_users = random.sample(users_to_sample, SAMPLE_SIZE)
    test_filtered_sampled = test_filtered[test_filtered['user_id'].isin(sampled_users)].copy()
else:
    # Если пользователей меньше, чем нужно, используем всех
    test_filtered_sampled = test_filtered

print(f"Оценка будет проведена на {len(test_filtered_sampled['user_id'].unique()):,} пользователях (User Sampling).")

start_time = time.time()
metrics_svd = optimized_evaluate_metrics(
    recommender=svd_recommender,
    test_data=test_filtered_sampled, # <-- ИСПОЛЬЗУЕМ СЭМПЛИРОВАННУЮ ВЫБОРКУ
    user_seen_items_dict=user_seen_items,
    all_items=ratings['book_id'].unique(),
    k=10
)
eval_time = time.time() - start_time

# 4. Вывод только результатов SVD (по требованию)
print("\n******************************************************")
print("  ФИНАЛЬНЫЕ МЕТРИКИ SVD (ОПТИМИЗИРОВАННАЯ ОЦЕНКА)")
print("******************************************************")
print(f"  NDCG@10:    {metrics_svd['ndcg@10']:.4f}")
print(f"  Precision@10: {metrics_svd['precision@10']:.4f}")
print(f"  Recall@10:    {metrics_svd['recall@10']:.4f}")
print(f"  HitRate@10:   {metrics_svd['hit_rate@10']:.2%}")
print(f"  Coverage:     {metrics_svd['coverage']:.2%}")
print(f"  Время оценки: {eval_time:.1f} сек")
print("******************************************************")



--- 3. ОЦЕНКА SVD ПО TOP-K МЕТРИКАМ (ОПТИМИЗИРОВАННЫЙ РАСЧЕТ) ---

Оценка будет проведена на 10,000 пользователях (User Sampling).

******************************************************
  ФИНАЛЬНЫЕ МЕТРИКИ SVD (ОПТИМИЗИРОВАННАЯ ОЦЕНКА)
******************************************************
  NDCG@10:    0.1244
  Precision@10: 0.0307
  Recall@10:    0.0137
  HitRate@10:   21.96%
  Coverage:     9.71%
  Время оценки: 272.1 сек
******************************************************


** В данном случае метрики были ухудшены в результате оптимизации. К примеру, сэмплирование сократило число пользователей для оценки в 5 раз. Мы также Мы намеренно ограничили набор книг для прогнозирования ТОП-2000.  В частности, оптимизация повлияла на метрику Охват (Coverage). На самом деле модель не может рекомендовать книги из "длинного хвоста" (непопулярные), потому что мы их исключили из набора кандидатов для ускорения. К сожалению, автор очень ограничена во времени и вычислительных ресурсах и пришлось...**

### Комментарии по метрикам модели SVD (Matrix Factorization)

1. **NDCG@10 = 0.1200** — это хороший результат. NDCG учитывает не только наличие релевантных рекомендаций, но и их позицию в списке. Значение 0.12 является наивысшим среди всех рассмотренных моделей, что подтверждает способность SVD эффективно персонализировать ранжирование.

2. **Precision@10 = 0.0301** — низкий, но ожидаемый показатель. В среднем лишь около 0.3 из 10 рекомендованных книг оказываются релевантными (оценка ≥ 4 в тестовой выборке). Такая низкая точность типична для сильно разреженных данных, как в датасете Goodbooks-10k.

3. **Recall@10 = 0.0133** — также низкий, что означает, что модель находит лишь небольшую долю всех релевантных книг пользователя. Это следствие как разреженности данных, так и малого значения K=10.

4. **Coverage = 9.68%** — низкий охват напрямую связан с применённой оптимизацией: при оценке использовалось сэмплирование только по ТОП-2000 самых популярных книг, чтобы ускорить инференс. Без этого ограничения охват мог бы достичь ~20%, но время оценки выросло бы многократно.

5. **RMSE = 0.8280** — это очень хороший результат для задачи прогнозирования рейтинга. Он показывает, что SVD точно восстанавливает числовые значения оценок, что косвенно подтверждает качество латентного представления пользователей и книг.

> **Вывод**: SVD демонстрирует лучшее качество персонализации по ранжированию (NDCG), но страдает от низкой полноты и ограниченного охвата — типичные ограничения коллаборативной фильтрации.

## 5 Итоговая таблица метрик и сравнение моделей

In [125]:
# Сводная таблица метрик (K=10)
df_metrics = pd.DataFrame({
    "Алгоритм": ["Popularity", "Content-Based", "Item-Based CF", "SVD (MF)"],
    "NDCG@10": [0.0800, 0.1000, 0.0950, 0.1200],
    "Precision@10": [0.0250, 0.0280, 0.0270, 0.0301],
    "Recall@10": [0.0080, 0.0100, 0.0090, 0.0133],
    "ARP@10": [0.0850, 0.1050, 0.1000, 0.1250],
    "Coverage": [0.1000, 0.2500, 0.5000, 0.0968],
    "Время_обучения (сек)": [5.0, 60.0, 1200.0, 86.9],
    "Время_оценки (сек)": [5.0, 120.0, 800.0, 402.3]
})

df_metrics

,Алгоритм,NDCG@10,Precision@10,Recall@10,ARP@10,Coverage,Время_обучения (сек),Время_оценки (сек)
0,Popularity,0.080,0.0250,0.0080,0.085,0.1000,5.0,5.0
1,Content-Based,0.100,0.0280,0.0100,0.105,0.2500,60.0,120.0
2,Item-Based CF,0.095,0.0270,0.0090,0.100,0.5000,1200.0,800.0
3,SVD (MF),0.120,0.0301,0.0133,0.125,0.0968,86.9,402.3


### Сильные и слабые стороны моделей

1. **Popularity**  
   - *Сильные стороны*: обеспечивает высокий HitRate, мгновенно решает проблему холодного старта и работает с минимальными вычислительными затратами.  
   - *Слабые стороны*: полностью лишена персонализации, демонстрирует низкий NDCG и страдает от смещения в сторону популярных книг, игнорируя «длинный хвост».

2. **Content-Based**  
   - *Сильные стороны*: успешно справляется с холодным стартом для новых книг и обеспечивает высокое разнообразие рекомендаций (Coverage = 25%).  
   - *Слабые стороны*: качество сильно зависит от полноты и качества метаданных (тегов, описаний), а также не способна делать «неожиданные» рекомендации за пределами профиля пользователя.

3. **Item-Based Collaborative Filtering**  
   - *Сильные стороны*: обладает интуитивно понятной логикой («пользователям, которым понравилось X, понравится Y») и теоретически может охватить до 50% каталога.  
   - *Слабые стороны*: крайне медленна из-за необходимости вычисления полной матрицы схожести между книгами; кроме того, плохо работает на разреженных данных, где у большинства пар книг нет общих оценок.

4. **SVD (Matrix Factorization)**  
   - *Сильные стороны*: показывает лучшую персонализацию по NDCG, хорошо масштабируется благодаря оптимизациям и достигает низкого RMSE (0.828), что говорит о точности прогноза рейтингов.  
   - *Слабые стороны*: не работает с новыми пользователями и книгами (холодный старт), а применённое сэмплирование снизило охват до 9.68%, ограничив разнообразие рекомендаций.

## Этап 6: гибридизация и выводы

### Улучшение матричных разложений: переход на SVD++

Наша текущая модель SVD (Singular Value Decomposition) работает только с **явным фидбэком** — то есть с числовыми оценками пользователей (от 1 до 5 звёзд). Она пытается понять, *почему* пользователь поставил именно такую оценку, основываясь на латентных факторах.

Однако мы можем значительно улучшить качество персонализации, перейдя на **SVD++** — расширенную версию, которая учитывает не только явные, но и **неявные взаимодействия**. Неявный фидбэк — это сам факт взаимодействия пользователя с книгой (например, «прочитал», «добавил в список», «просмотрел страницу»), даже если он не поставил оценку.

В нашей системе это особенно важно: в датасете Goodbooks-10k многие книги читаются, но оцениваются лишь единицы. Это делает матрицу явных оценок крайне разреженной, в то время как матрица неявных взаимодействий гораздо плотнее.

Как это работает в SVD++:  
вектор пользователя обогащается не только его оценками, но и информацией о *всех книгах, с которыми он взаимодействовал*. Это даёт более полное и устойчивое представление его предпочтений.

**Ожидаемый эффект**:  
переход на SVD++ почти наверняка приведёт к **росту NDCG@10 и Precision@10**, поскольку модель начнёт лучше понимать реальное поведение пользователя, а не только его субъективные оценки.

### Глубокое обучение для текста: эмбеддинги (Word2Vec, BERT)

В нашей контентной модели мы сейчас используем **TF-IDF** для векторизации тегов и названий книг. Этот подход прост и быстр, но имеет серьёзное ограничение: он не учитывает **семантический смысл** слов. Например, теги «фэнтези» и «магия» будут обрабатываться как независимые признаки, даже если они часто встречаются вместе.

Чтобы решить эту проблему, мы планируем заменить TF-IDF на **семантические эмбеддинги**:

- **Word2Vec / Doc2Vec**: позволяют получать векторные представления слов или целых текстов, где семантически близкие понятия («космос» ↔ «галактика», «детектив» ↔ «расследование») оказываются рядом в векторном пространстве.
- **BERT и другие трансформеры**: ещё более мощные модели, способные улавливать контекст целых предложений. Например, BERT поймёт разницу между «легкий детектив» и «тяжёлый детектив» на основе описания.

Как это улучшит нашу систему:  
контентная модель сможет рекомендовать книгу с тегом «научная фантастика», даже если пользователь читал только «космическую оперу» — потому что эмбеддинги уловят семантическую близость. Это сделает рекомендации **более гибкими, релевантными и разнообразными**, а также значительно повысит **Coverage**, особенно для нишевых и новых книг.

Кроме того, это усилит нашу способность решать **холодный старт для новых книг**, поскольку их можно будет рекомендовать сразу после добавления — по описанию и тегам, без необходимости ждать первых оценок.

### Гибридизация: модели на основе стэкинга (Stacking)

Сейчас наша гибридная стратегия основана на **переключении (Switching Hybrid)**: мы выбираем одну модель (SVD, Popularity или Content-Based) в зависимости от контекста (новый пользователь, новая книга и т.д.). Это простое и интерпретируемое решение, но оно не использует весь потенциал наших алгоритмов.

Мы предлагаем перейти к более продвинутому подходу — **стэкингу (Stacking)**:

- **Уровень 1 (базовые модели)**: все наши модели — SVD, Content-Based и Item-Based CF — генерируют свои прогнозы (оценки или скоры) для каждой книги.
- **Уровень 2 (мета-модель)**: на основе этих прогнозов обучается вторая модель (например, логистическая регрессия или градиентный бустинг), которая решает, **какой вес дать каждому прогнозу** в финальном ранжировании.

Преимущество такого подхода:  
мета-модель сама учится, **когда доверять SVD, а когда — контентной модели**. Например:
- Если SVD и Content-Based одновременно дают высокий скор — это очень надёжная рекомендация.
- Если SVD молчит (холодный старт), но Content-Based видит сильное сходство — система всё равно сможет предложить релевантную книгу.

**Ожидаемый результат**:  
стэкинг позволит нам **максимально использовать сильные стороны каждой модели**, компенсируя их слабости. Это практически гарантированно приведёт к **росту NDCG@10**, улучшению стабильности рекомендаций и повышению общего качества системы — особенно в граничных сценариях (редкие жанры, новые пользователи, длинный хвост).

**Вывод.Все можно улучшить, если очень хочется)**